In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e2/sample_submission.csv
/kaggle/input/playground-series-s6e2/train.csv
/kaggle/input/playground-series-s6e2/test.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s6e2/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s6e2/test.csv")
train.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [4]:
test_ids = test['id']
test_ids.head()

0    630000
1    630001
2    630002
3    630003
4    630004
Name: id, dtype: int64

In [5]:
train['Heart Disease'] = train['Heart Disease'].map({
    'Presence' : 1,
    'Absence' : 0
})
train = train.drop('id' , axis=1)
test = test.drop('id' , axis=1)

In [6]:
train['Heart Disease'].value_counts(normalize=True)

Heart Disease
0    0.55166
1    0.44834
Name: proportion, dtype: float64

In [7]:
train['Heart Disease'] = train['Heart Disease'].astype(int)

In [8]:
X = train.drop(columns=['Heart Disease'] , axis=1)
y = train['Heart Disease']


X.shape , y.shape , test.shape

((630000, 13), (630000,), (270000, 13))

In [9]:
from sklearn.model_selection import StratifiedKFold , cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier


In [10]:
cat_model_tuned = CatBoostClassifier(
    iterations = 1500,
    depth = 7,
    learning_rate = 0.03,
    l2_leaf_reg = 5,
    loss_function = 'Logloss',
    eval_metric ='AUC',
    random_state = 42,
    verbose = 0,
    task_type='GPU'
)

In [11]:
cv = StratifiedKFold(n_splits = 5 , shuffle = True ,random_state = 42)

cv_scores = cross_val_score(
    cat_model_tuned,
    X,
    y,
    cv = cv,
    scoring = 'roc_auc'
)

print("CV AUC Scores", cv_scores)
print("Mean CV AUC Scores",cv_scores.mean())

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


CV AUC Scores [0.95559794 0.95459962 0.95540399 0.95493351 0.95576974]
Mean CV AUC Scores 0.9552609586404912


In [12]:
cat_model_tuned.fit(X,y)

Default metric period is 5 because AUC is/are not implemented for GPU


In [13]:
test_preds = cat_model_tuned.predict_proba(test)[:,1]


#    **Thanks for checking out my notebook**

this is my **first kaggle competation** ,and i focused on building a clean and understandable pipeline.

i'd love to hear your **Feedback or tips** to improve my workflow. Let's connect and learn together